In [ ]:
!pip install -q google-generativeai

In [ ]:
import os
import time
import textwrap
import google.generativeai as genai

from google.colab import userdata

genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))

In [ ]:
!curl -o gemini.pdf https://storage.googleapis.com/cloud-samples-data/generative-ai/pdf/2403.05530.pdf

In [ ]:
!curl -o base_model.pdf https://arxiv.org/pdf/2312.01552

In [ ]:
# Upload the file and print a confirmation
sample_file = genai.upload_file(path="gemini.pdf",
                                display_name="Gemini 1.5 PDF")

print(f"Uploaded file '{sample_file.display_name}' as: {sample_file.uri}")

## Verify PDF file upload and get metadata
You can verify the API successfully stored the uploaded file and get its metadata by calling files.get through the SDK. Only the name (and by extension, the uri) are unique. Use display_name to identify files only if you manage uniqueness yourself.

In [ ]:
file = genai.get_file(name=sample_file.name)
print(f"Retrieved file '{file.display_name}' as: {sample_file.uri}")

## Prompt the Gemini API with the uploaded documents
After uploading the file, you can make GenerateContent requests that reference the File API URI. Select the generative model and provide it with a text prompt and the uploaded document:

In [ ]:
# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro")

# Prompt the model with text and the previously uploaded image.
response = model.generate_content([sample_file, "Can you summarize this document as a bulleted list?"])

print(response.text)

In [ ]:
print(response.usage_metadata)

In [ ]:
# Prompt the model with text and the previously uploaded image.
response = model.generate_content([sample_file, "Can you explain Figure 9 in the paper?"])

print(response.text)

In [ ]:
print(response.usage_metadata)

In [ ]:
# Prompt the model with text and the previously uploaded image.
response = model.generate_content([sample_file, "Can you describe the scene in Figure 15 in details? How many people do you see in the image? and what is the caption of the image"])

print(response.text)

In [ ]:
textwrap.wrap(response.text, width=80)

## Working with Multiple Files

In [ ]:
# Upload the file and print a confirmation
base_model_file = genai.upload_file(path="base_model.pdf",
                                    display_name="Base Model PDF")

print(f"Uploaded file '{base_model_file.display_name}' as: {base_model_file.uri}")

In [ ]:
file = genai.get_file(name=base_model_file.name)
print(f"Retrieved file '{file.display_name}' as: {file.uri}")

In [ ]:
# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-flash")

prompt = "Summarize the differences between the thesis statements for these documents."

response = model.generate_content([prompt, sample_file, base_model_file, ])

In [ ]:
textwrap.wrap(response.text, width=120)

In [ ]:
prompt = "Do you think the URIAL approach has validity? Can you give me counter arguments?"
response = model.generate_content([prompt, sample_file, base_model_file, ])

In [ ]:
textwrap.wrap(response.text, width=120)

In [ ]:
print(response.text)

## List files
You can list all files uploaded using the File API and their URIs using files.list_files():

In [ ]:
# List all files
for file in genai.list_files():
    print(f"{file.display_name}, URI: {file.uri}")

### Adding Context Cache

In [ ]:
import google.generativeai as genai
from google.generativeai import caching
import datetime

In [ ]:
# Create a cache with a 5 minute TTL
cache = caching.CachedContent.create(
    model='models/gemini-1.5-flash-001',
    display_name='PDF-file', # used to identify the cache
    system_instruction=(
        'You are an expert PDF file analyzer, and your job is to answer '
        'the user\'s query based on the PDF file you have access to.'
    ),
    contents=[sample_file, ],
    ttl=datetime.timedelta(minutes=15)
)

In [ ]:
# Construct a GenerativeModel which uses the created cache.
model = genai.GenerativeModel.from_cached_content(cached_content=cache)

# Query the model
response = model.generate_content([(
    'What is the title of the paper?'
    'Who are the authors? '
    'What are the major contributions of the paper accordig to the authors?'
)])

print(response.usage_metadata)

print(response.text)

In [ ]:
# Construct a GenerativeModel which uses the created cache.
model = genai.GenerativeModel.from_cached_content(cached_content=cache)

# Query the model
response = model.generate_content([(
    'What is the title of the paper?'
    'Who are the authors? provide a list '
    'What are the major contributions of the paper accordig to the authors?'
)])

print(response.usage_metadata)

print(response.text)

In [ ]:
for c in caching.CachedContent.list():
  print(c)